In [22]:
# @ written by shubham bansal:
import pandas as pd
import  numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
import pygmaps
from scipy.spatial import ConvexHull
from datetime import datetime
# @ written by shubham bansal:
from haversine import haversine
%matplotlib inline

# print('heLlo')



In [23]:
data=pd.read_csv('Data_july_1.csv')
S_data=data.sort_values('IST_TIME')
Data_array=S_data.as_matrix()
data.columns
Data_array.shape[0]


# np.where(df[''].map(len) > df['b'].map(len), df['a'].map(len), df['b'].map(len))

4924

In [24]:
S_lat=Data_array[0][1]
S_long=Data_array[0][2]
Start_Acc=Data_array[0][6]
Time=Data_array[0][5]
size=Data_array.shape[0]
Data_Train=pd.DataFrame(columns=['Latitude','Longitude','Acc_distance','Time'])
Data_Train=Data_Train.append({'Latitude':S_lat,'Longitude':S_long,'Acc_distance':Start_Acc,'Time':Time},ignore_index=True)


## SLOT DIVISION


In [25]:


for i in range(0,1127):
    if abs(Start_Acc-Data_array[i][6]) >=100 :   
         Start_Acc=Data_array[i][6]
         Data_Train=Data_Train.append({'Latitude':Data_array[i][1],'Longitude':Data_array[i][2],'Acc_distance':Start_Acc,'Time':Data_array[i][5]},ignore_index=True)
    else :
        continue




In [38]:
Data_Train.to_csv('Slot_file.csv',index=False)

In [26]:
Slot_Matrix=Data_Train.as_matrix()


## Addition for Time in sec Column 

In [27]:
# Data_making=pd.read_csv('150218661.csv')

In [28]:
# Data_Treatment=Data_making[['LAT','LONGITUDE','IST_TIME','IST_DATE','SPEED_KMPH']].copy()
# Data_Treatment=Data_Treatment.drop_duplicates(subset=['LAT','LONGITUDE'], inplace=False)
# Data_Treatment['Time_in_sec'] = pd.Series(np.random.randn(Data_Treatment.shape[0]), index=Data_Treatment.index)

In [29]:
# Data_Treatment = Data_Treatment.as_matrix()
# for i in range(0,Data_Treatment.shape[0]):
#     s=Data_Treatment[i][2]
#     Total_day_time=sum(int(i) * 60**index for index, i in enumerate(s.split(":")[::-1]))
#     Data_Treatment[i][5]=Total_day_time
    

In [30]:
# Data_array_size=Data_Treatment.shape[0]
# Slot_matrix_size=Slot_Matrix.shape[0]


# count=np.zeros((Slot_matrix_size-1))
# speed=np.zeros(Slot_matrix_size-1)
# avg = np.zeros(Slot_matrix_size-1)

In [31]:
# count

In [32]:
# speed

In [33]:
def data_processor(filename,newfilename):
    
#     Data_making=pd.read_csv(filename)
    
    Data_Treatment=filename[['LAT','LONGITUDE','IST_TIME','IST_DATE','SPEED_KMPH']].copy()
    #print(Data_Treatment)
    Data_Treatment=Data_Treatment.drop_duplicates(subset=['LAT','LONGITUDE'], inplace=False)
    Data_Treatment['Time_in_sec'] = pd.Series(np.random.randn(Data_Treatment.shape[0]), index=Data_Treatment.index)

    #appending columns for ml application later ->
    Data_Treatment['Slot_number'] = pd.Series(np.random.randn(Data_Treatment.shape[0]), index=Data_Treatment.index)
    
    Data_Treatment = Data_Treatment.as_matrix()
    
    for i in range(0,Data_Treatment.shape[0]):
        s=Data_Treatment[i][2]
        Total_day_time=sum(int(i) * 60**index for index, i in enumerate(s.split(":")[::-1]))
        Data_Treatment[i][5]=Total_day_time
        Data_Treatment[i][6]=-1
#     print(Data_Treatment)
    
    Data_array_size=Data_Treatment.shape[0]
    Slot_matrix_size=Slot_Matrix.shape[0]
    count=np.zeros((Slot_matrix_size-1))
    speed=np.zeros(Slot_matrix_size-1)
    avg = np.zeros(Slot_matrix_size-1)
    
    
    for i in range(0,Data_array_size):
    #     print(i)
        for j in range(1,Slot_matrix_size):
            flag_lat=0;
            flag_long=0;
    #         print(j)
    #         print(Slot_Matrix[j-1,0], Data_Treatment[i,0],Slot_Matrix[j,0],Data_Treatment[i,0]-Slot_Matrix[j-1,0],Data_Treatment[i,0]-Slot_Matrix[j,0])
    #         print(Slot_Matrix[j-1,1],Data_Treatment[i,1],Slot_Matrix[j,1],Data_Treatment[i,1]-Slot_Matrix[j-1,1],Data_Treatment[i,1]-Slot_Matrix[j,1])
            x_lat=Data_Treatment[i,0]-Slot_Matrix[j-1,0]
            y_lat=Data_Treatment[i,0]-Slot_Matrix[j,0]
            x_long=Data_Treatment[i,1]-Slot_Matrix[j-1,1]
            y_long=Data_Treatment[i,1]-Slot_Matrix[j,1]
            if(x_lat>0):
                if(y_lat<0):
                    flag_lat=1
    #                 print("lat opp mila",x_lat,y_lat)
            if(x_lat<0):
                if(y_lat>0):
                    flag_lat=1
    #                 print("lat opp mila",x_lat,y_lat)

            if(x_long>0):
                if(y_long<0):
                    flag_long=1
    #                 print("long opp mila",x_long,y_long)

            if(x_long<0):
                if(y_long>0):
                    flag_long=1
    #                 print("long opp mila",x_long,y_long)       

    #         if ((((Data_Treatment[i][0]- Slot_Matrix[j][0])>0 and (Data_Treatment[i][0]-Slot_Matrix[j-1][0])<0) \
    #         or((Data_Treatment[i][0]-Slot_Matrix[j][0])<0 and (Data_Treatment[i][0]-Slot_Matrix[j-1][0])>0))\
    #         or\
    #         (((Data_Treatment[i][0]-Slot_Matrix[j][0])>0 and (Data_Treatment[i][1]-Slot_Matrix[j-1][1])<0)\
    #         or((Data_Treatment[i][0]-Slot_Matrix[j][0])<0 and (Data_Treatment[i][1]-Slot_Matrix[j-1][1])>0))\
    #            ):
            #if((((Data_Treatment[i,0]-Slot_Matrix[j-1,0])>0 and (Data_Treatment[i,0]-Slot_Matrix[j,0])<0) or ((Data_Treatment[i,0]-Slot_Matrix[j-1,0])<0 and (Data_Treatment[i,0]-Slot_Matrix[j,0])>0)) or (((Data_Treatment[i,1]-Slot_Matrix[j-1,1])>0 and (Data_Treatment[i,1]-Slot_Matrix[j,1])<0) or ((Data_Treatment[i,1]-Slot_Matrix[j-1,1])<0 and (Data_Treatment[i,1]-Slot_Matrix[j,1])>0))):

            if(flag_lat==1 or flag_long==1):
                #print("*************************************************************************************************************************************")
                count[j-1]=count[j-1]+1.0
                speed[j-1]=speed[j-1]+Data_Treatment[i][4]   
                Data_Treatment[i][6] = j-1
                break
    
    for i in range(0, Slot_matrix_size-1):
        if(count[i]==0):
            avg[i] = 0
            continue
        avg[i] = speed[i]/count[i]
    
#     print(count)
#     print(speed)
#     print(avg)
    x=pd.DataFrame(Data_Treatment,columns=['LAT','LONGITUDE','IST_TIME','IST_DATE','SPEED_KMPH','Time_sec','Slot_no'])
#     print(Data_Treatment)
    x.to_csv(newfilename,index=False)

    
#     congestion_flag = np.zeros(Slot_matrix_size-1)
    
#     for i in range(0, Slot_matrix_size-1):
#         if(avg[i]==0.0): 
#             congestion_flag[i] = 0 #no congestion
#             continue
#         if(avg[i]>=10.0):
#             congestion_flag[i] = 1 #mild congestion
#         if(avg[i] >= 5.0 and avg[i] <10.0):
#             congestion_flag[i] = 2 #severe congestion
#         if(avg[i] < 5.0):
#             congestion_flag[i] = 3 #heavy congestion
        
#     print(congestion_flag)
    

In [34]:
# for i in range(0, Slot_matrix_size-1):
#     if(count[i]==0):
#         avg[i] = 0
#         continue
#     avg[i] = speed[i]/count[i]
    

# avg


In [35]:
def Load_file(filename, newfilename):
    Data_Train=pd.read_csv(filename)
#     Data_Train_final=Data_prepration(Data_Train, newfilename)
    Data_Train_final=data_processor(Data_Train, newfilename)

In [36]:
import os
root = '/home/b/Desktop/Traffic_prediction/gis_project/Data_route_wise/356_EW'
for subdir, dirnames, filenames in os.walk(root):
    for filename in filenames:
        if  filename.startswith('2016'):#     print(subdir, filename)
            folders = subdir.split(sep='/')
            newfilename = folders[7]+'_'+folders[8]+'_'+filename
      #  print(os.path.join(subdir,filename),newfilename)
            print(newfilename)
            Load_file(os.path.join(subdir,filename), newfilename)

356_EW_6july_2016-07-06.csv
356_EW_10july_2016-07-10.csv
356_EW_8july_2016-07-08.csv
356_EW_16july_2016-07-16.csv
356_EW_15july_2016-07-15.csv
356_EW_31july_2016-07-31.csv
356_EW_17july_2016-07-17.csv
356_EW_20july_2016-07-20.csv
356_EW_4july_2016-07-04.csv
356_EW_5july_2016-07-05.csv
356_EW_29july_2016-07-29.csv
356_EW_26july_2016-07-26.csv
356_EW_18july_2016-07-18.csv
356_EW_3july_2016-07-03.csv
356_EW_25july_2016-07-25.csv
356_EW_24july_2016-07-24.csv
356_EW_21july_2016-07-21.csv
356_EW_9july_2016-07-09.csv
356_EW_12july_2016-07-12.csv
356_EW_28july_2016-07-28.csv
356_EW_27july_2016-07-27.csv
356_EW_19july_2016-07-19.csv
356_EW_1july_2016-07-01.csv
356_EW_22july_2016-07-22.csv
356_EW_14july_2016-07-14.csv
356_EW_7july_2016-07-07.csv
356_EW_30july_2016-07-30.csv
356_EW_13july_2016-07-13.csv
356_EW_11july_2016-07-11.csv
356_EW_23july_2016-07-23.csv


## Plotting the curve

In [31]:
mymap_1_111=pygmaps.pygmaps(12.91735500,77.62260400,180)
# data=pd.read_csv('Data_july_1.csv')
# lat_long_time=data[['LAT','LONGITUDE','IST_TIME']].copy()
# print(data_with_out.shape[0])
# data_with_out_dupliacte = data_with_out.as_matrix(columns=['LAT','LONGITUDE','Time'])


count=0

for j in range(0,Slot_matrix_size):
#        if( (data_with_out_dupliacte[j][2]>'07:00:00') and (data_with_out_dupliacte[j][2]<'08:30:00') ):
          mymap_1_111.addpoint(Slot_Matrix[j][0],Slot_Matrix[j][1],"#FF0000")


# for i  in range(0,19):     
#     mymap_1_111.addpoint(Data_Treatment[i][0],Data_Treatment[i][1],"#00ffff")




# mymap_1_111.addpoint(Data_Treatment[13][0],Data_Treatment[13][1],)
# mymap_1_111.addpoint(Data_Treatment[13][0],Data_Treatment[13][1],)
# mymap_1_111.addpoint(Data_Treatment[13][0],Data_Treatment[13][1],)





mymap_1_111.draw('a.html')





















11.99179996235805

In [ ]:
import os
root = '/home/b/Desktop/Traffic_prediction/gis_project/Data_route_wise/356_EW/1july'
for subdir, dirnames, filenames in os.walk(root):
    for filename in filenames:
        if filename.endswith('-checkpoint.csv') or filename.startswith('2016'):
            continue
#         print(subdir, filename)
        folders = subdir.split(sep='/')
        newfilename = folders[7]+'_'+folders[8]+'_'+filename
      #  print(os.path.join(subdir,filename),newfilename)
        print(newfilename)
        Load_file(os.path.join(subdir,filename), newfilename)